In [77]:
import pymongo
from pymongo import MongoClient
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker
import pandas as pd
import decimal
import json

In [78]:
client = MongoClient('localhost', 27017)

In [79]:
db = client.zams

In [80]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'zams')

In [81]:
collection = db.employees

In [82]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'zams'), 'employees')

In [83]:
password = 'insert your password' #replace with your password
db_name = 'zams'

In [84]:
#connecting to MySQL database
engine = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)
Session = sessionmaker(bind=engine)
session = Session()

In [85]:
# Creating SQLAlchemy tables
employees_table = Table('employees', MetaData(), autoload_with=engine)
departments_table = Table('departments', MetaData(), autoload_with=engine)
salaries_table = Table('salaries', MetaData(), autoload_with=engine)
taxes_table = Table('taxes', MetaData(), autoload_with=engine)

In [86]:
#Querying data from MySQL
employees = session.query(employees_table).all()

In [88]:
# Iterating through employees and merging related information
for emp in employees:
    employee_doc = {
        "_id": emp.Employee_ID,
        "first_name": emp.First_Name,
        "last_name": emp.Last_Name,
        "hire_date": emp.Hire_Date,
        "birth_date": emp.Birth_Date,
        "email": emp.Email,
        "phone": emp.Phone,
        "address": emp.Address,
        "department": {},
        "salaries": [], #adding array of salaries that each represent a given pay cycle
        "taxes": {}
    }
    
    # Querying department table
    department = session.query(departments_table).filter_by(Depart_ID=emp.Depart_ID).first()
    employee_doc["department"] = {
        "department_id": department.Depart_ID,
        "department_name": department.Depart_Name,
        "floor": department.Floor,
        "year_established": department.Year_Established,
        "budget_millions": department.Budget_millions,
        "size": department.Size,
        "department_head": department.Depart_Head,
        "primary_function": department.Primary_Func
    }
    
    # Querying salary information for multiple months
    salaries = session.query(salaries_table).filter_by(Employee_ID=emp.Employee_ID).all()
    for salary in salaries:
        # Converting Decimal values to float since MongoDB can't handle the decimal data type
        salary_data = {
            "salary_id": salary.Salary_ID,
            "pay_date": salary.Pay_Date,
            "total_work_time": float(salary.Total_Work_Time),
            "base_salary_rate": float(salary.Base_Salary_Rate),
            "overtime_rate": float(salary.Overtime_Rate),
            "quarterly_bonus": float(salary.Quarterly_Bonus),
            "gross_pay": float(salary.Gross_Pay),
            "tax_deduction": float(salary.Tax_Deduction),
            "social_security_deduction": float(salary.SocialSecurity_Deduction),
            "payment_method": salary.Payment_Method
        }
        employee_doc["salaries"].append(salary_data)
    
    # Querying tax information for each employee
    tax = session.query(taxes_table).filter_by(Employee_ID=emp.Employee_ID).first()
    if tax:
        # Convert Decimal values to float again
        tax_data = {
            "tax_id": tax.Tax_ID,
            "fed_tax_rate": float(tax.FedTax_Rate),
            "state_tax_rate": float(tax.StateTax_Rate),
            "fed_withhold": float(tax.Fed_Withhold),
            "state_withhold": float(tax.State_Withhold),
            "social_security_tax": float(tax.SocialSecuity_Tax),
            "med_tax": float(tax.Med_Tax),
            "exemption": float(tax.Exemption),
            "total_tax_deduction": float(tax.Total_Tax_Deduction),
            "overall_deduction": float(tax.Overall_Deduction)
        }
        employee_doc["taxes"] = tax_data
        
    
    # Insert the nested document into the MongoDB collection
    collection.insert_one(employee_doc)

# Close connections
# session.close()
# client.close()